In [45]:
import csv
import datetime
import gzip
import random
from collections import defaultdict

import numpy as np
import scipy
import scipy.optimize
import tensorflow as tf
from sklearn import linear_model
import pandas as pd
from datetime import datetime

timeThreshold = 60

def readGz(path):
    for l in gzip.open(path, "rt"):
        yield l

In [3]:
def AUCu(model, u, N):
    win = 0
    if N > len(interactionsTestPerUser[u]):
        N = len(interactionsTestPerUser[u])
    positive = random.sample(interactionsTestPerUser[u],N)
    negative = random.sample(itemSet,N)
    for (i,j),k in zip(positive,negative):
        sp = model.predict(userIDs[u], itemIDs[i], itemIDs[j]).numpy()
        sn = model.predict(userIDs[u], itemIDs[k], itemIDs[j]).numpy()
        if sp > sn:
            win += 1
    return win/N

In [4]:
def AUC(model):
    av = []
    for u in interactionsTestPerUser:
        av.append(AUCu(model, u, 10))
    return sum(av) / len(av)

In [5]:
interaction = []
for l in readGz("Behance_appreciate_1M.gz"):
    items = [i.replace('\n', '') for i in l.split(' ')]
    interaction.append(items)

In [6]:
interaction[:5]

[['276633', '01588231', '1307583271'],
 ['1238354', '01529213', '1307583273'],
 ['165550', '00485000', '1307583337'],
 ['2173258', '00776972', '1307583340'],
 ['165550', '00158226', '1307583406']]

In [7]:
len(interaction)

1000000

In [8]:
interactionPerUser = defaultdict(list)
interactionCountPerUser = defaultdict(int)
for d in interaction:
    u = d[0]
    i = d[1]
    ts = int(d[2])
    interactionPerUser[u].append((i, ts))
    interactionCountPerUser[u] += 1

In [9]:
len(interactionPerUser)

63497

In [10]:
sum([interactionCountPerUser[u] >= 3 for u in interactionCountPerUser]) / len(interactionCountPerUser)

0.616895286391483

In [11]:
# Train/Val/Test Split
interactionsTrainPerUser = defaultdict(list)
interactionsTestPerUser = defaultdict(list)
interactionsValPerUser = defaultdict(list)
for u in interactionPerUser:
    if interactionCountPerUser[u] >= 3:
        sort_int = sorted(interactionPerUser[u], key = lambda x: x[1])
        interactionsTestPerUser[u] = sort_int[-1]
        interactionsValPerUser[u] = sort_int[-2]
        interactionsTrainPerUser[u] = sort_int[:-2]

In [12]:
interactionsTrainPerUser['276633']
len(interactionsTrainPerUser)

39171

In [73]:
# mostPopular = [(itemCount[d], d) for d in itemCount]
# mostPopular.sort(reverse = True)

In [44]:
# mostPopular[0][1]

### Popularity

In [46]:
# itemCount = defaultdict(int)
# totalCount = defaultdict(int)
timeCount = defaultdict(lambda : defaultdict(int))
for d in interactionsTrainPerUser:
    l = interactionsTrainPerUser[d]
    for item, ts in l:
        ts = ts // (timeThreshold*60)
        timeCount[ts][item] += 1
#         itemCount[(item, ts)] += 1
#         totalCount[ts] += 1

In [172]:
# timeCount[1465611]

defaultdict(int,
            {'02245390': 1,
             '02333640': 1,
             '01969117': 1,
             '02345212': 20,
             '00750476': 1,
             '01384927': 1,
             '01609277': 1,
             '00683015': 1,
             '00933954': 1,
             '01011009': 1,
             '00980697': 1,
             '01461631': 1,
             '00773933': 1,
             '02324622': 4,
             '01265625': 1,
             '01763950': 6,
             '02345150': 1,
             '02344912': 1,
             '02344768': 1,
             '01786606': 2,
             '01764508': 2,
             '01971913': 1,
             '00783161': 1,
             '02345536': 2,
             '01956947': 1,
             '00412167': 1,
             '00421600': 1,
             '00550067': 1,
             '00529316': 1,
             '00048787': 1,
             '00518601': 1,
             '01486677': 1,
             '00381691': 1,
             '01531061': 1,
             '02330340': 1,
  

In [50]:
def popularity_predict(ts, N):
    index = ts // (timeThreshold * 60)
#     print(index)
    popularItem = timeCount[index]
    mostPopular = [(popularItem[d], d) for d in popularItem]
    mostPopular.sort(reverse = True)
    if len(mostPopular) < N:
        N = len(mostPopular)
#     print(mostPopular[:N])
    recommend = [d[1] for d in mostPopular[:N]]
    
    return recommend
#     if len(mostPopular) == 0:
#         return 0
#     return mostPopular[0][1]

In [119]:
# rec = popularity_predict(1465611 * (timeThreshold * 60))
# rec

'02345212'

In [51]:
def popularity_AUCu(u, N):
    win = 0
    item = interactionsTestPerUser[u][0]
    ts = interactionsTestPerUser[u][1]
    rec = popularity_predict(ts, N)
    if item in rec:
        win += 1
#     if rec == item:
#         print('YEs')
#         win += 1       
    return win

In [52]:
# Calculate AUC
av = []
for u in interactionsTestPerUser:
    av.append(popularity_AUCu(u, 10))
AUC = sum(av) / len(av)
print("AUC of popularity: ", AUC)

AUC of popularity:  0.12236093028005413


### Time Threshold

In [8]:
df = pd.DataFrame(interaction, columns = ['user_id', 'item_id', 'timestamp'])
df.head()

,user_id,item_id,timestamp
0,276633,01588231,1307583271
1,1238354,01529213,1307583273
2,165550,00485000,1307583337
3,2173258,00776972,1307583340
4,165550,00158226,1307583406


In [10]:
a = df.timestamp.value_counts()
a[a>1]

1311594039    7
1309770025    6
1308921052    6
1310063412    5
1315319115    5
             ..
1311097371    2
1311725586    2
1313749874    2
1312990068    2
1314007758    2
Name: timestamp, Length: 48381, dtype: int64

In [15]:
dt = datetime.fromtimestamp(1307583271)
dt
print(dt.strftime("%Y-%m-%d %H:%M:%S"))

2011-06-09 01:34:31


In [37]:
print(1307583271 // 300, 1307583337 // 300, 1307583707//300, 1307583567//300)

4358610 4358611 4358612 4358611


In [17]:
df['timestamp'] = df.timestamp.astype("int")
df['time'] = df.timestamp.apply(lambda x: datetime.fromtimestamp(x).strftime("%Y-%m-%d %H:%M:%S"))
df.head()

,user_id,item_id,timestamp,time
0,276633,01588231,1307583271,2011-06-09 01:34:31
1,1238354,01529213,1307583273,2011-06-09 01:34:33
2,165550,00485000,1307583337,2011-06-09 01:35:37
3,2173258,00776972,1307583340,2011-06-09 01:35:40
4,165550,00158226,1307583406,2011-06-09 01:36:46


In [33]:
d = datetime(1970, 1, 1, 0, 5, 0)
print(d.strftime("%Y-%m-%d %H:%M:%S"))
timestamp = datetime.timestamp(d)
print(timestamp)

1970-01-01 00:05:00
300.0


In [40]:
df['time_5'] = df.timestamp // (5 * 60)
df['time_10'] = df.timestamp // (10 * 60)
df['time_15'] = df.timestamp // (15 * 60)
df['time_20'] = df.timestamp // (20 * 60)
df['time_30'] = df.timestamp // (30 * 60)
df['time_60'] = df.timestamp // (60 * 60)

In [48]:
b = df.time_5.value_counts()
print(max(b), min(b), len(b))

b = df.time_10.value_counts()
print(max(b), min(b), len(b))

b = df.time_15.value_counts()
print(max(b), min(b), len(b))

b = df.time_20.value_counts()
print(max(b), min(b), len(b))

b = df.time_30.value_counts()
print(max(b), min(b), len(b))

b = df.time_60.value_counts()
print(max(b), min(b), len(b))

86 1 45550
148 1 22781
205 2 15187
270 7 11391
394 18 7594
749 51 3798


In [62]:
b = df.time_15.value_counts()
sum(b <= 10) / len(b)

0.0022387568315006255